# Intermediate Tutorial: Composition and Navigation

This tutorial covers composing prompts from smaller pieces, navigating nested structures, and exporting data.

In [1]:
from t_prompts import prompt

## Nested Prompts

You can build larger prompts by composing smaller `StructuredPrompt` objects together.

In [2]:
# Build prompts from smaller pieces
system_msg = "You are a helpful assistant."
user_query = "What is Python?"

p_system = prompt(t"{system_msg:system}")
p_user = prompt(t"User: {user_query:query}")

# Compose into larger prompt
p_full = prompt(t"{p_system:sys} {p_user:usr}")

# Renders correctly
print(str(p_full))

You are a helpful assistant. User: What is Python?


## Navigating Nested Structures

Access nested interpolations using chained subscript operations.

In [3]:
# Navigate the tree
print(f"System message: {p_full['sys']['system'].value}")
print(f"User query: {p_full['usr']['query'].value}")

# When accessing a nested prompt node, it returns a StructuredPrompt
nested_node = p_full['sys']
print(f"\nNested node type: {type(nested_node).__name__}")
print(f"Has 'system' key: {'system' in nested_node}")
print(f"Rendered: {str(nested_node)}")

System message: You are a helpful assistant.
User query: What is Python?

Nested node type: StructuredPrompt
Has 'system' key: True
Rendered: You are a helpful assistant.


## Conversions: !s, !r, !a

t-strings support conversion flags from Python's string formatting.

In [4]:
text = "Hello\nWorld"

# !s: str() conversion (default)
p_s = prompt(t"{text!s:s}")
print(f"!s: {str(p_s)}")

# !r: repr() conversion
p_r = prompt(t"{text!r:r}")
print(f"!r: {str(p_r)}")

# !a: ascii() conversion
emoji = "Hello 👋"
p_a = prompt(t"{emoji!a:a}")
print(f"!a: {str(p_a)}")

# Conversion is preserved in metadata
print(f"\nConversion for 'r': {p_r['r'].conversion}")

!s: Hello
World
!r: 'Hello\nWorld'
!a: 'Hello \U0001f44b'

Conversion for 'r': r


## Mapping Protocol

`StructuredPrompt` implements the mapping protocol: `keys()`, `values()`, `items()`, `get()`, etc.

In [5]:
name = "Alice"
age = "30"
city = "NYC"

p = prompt(t"Name: {name:n}, Age: {age:a}, City: {city:c}")

# Keys
print(f"Keys: {list(p.keys())}")

# Values (returns interpolation nodes)
print("\nValues:")
for node in p.values():
    print(f"  {node.key}: {node.value}")

# Items
print("\nItems:")
for key, node in p.items():
    print(f"  {key} -> {node.expression} = {node.value}")

# get() with default
print(f"\nget('n'): {p.get('n').value}")
print(f"get('missing'): {p.get('missing')}")

Keys: ['n', 'a', 'c']

Values:
  n: Alice
  a: 30
  c: NYC

Items:
  n -> name = Alice
  a -> age = 30
  c -> city = NYC

get('n'): Alice
get('missing'): None


## Exporting to JSON

Use `toJSON()` to export the prompt structure as JSON.

In [6]:
import json

context = "User is Alice"
instructions = "Be concise"

p = prompt(t"Context: {context:ctx}. {instructions:inst}")

# Export to JSON
json_data = p.toJSON()
print(json.dumps(json_data, indent=2))

{
  "prompt_id": "c048387c-480d-4bc8-bb98-1bed69ca452a",
  "children": [
    {
      "type": "static",
      "id": "67eb6a9b-3b10-4222-a756-1a695991af5f",
      "parent_id": "c048387c-480d-4bc8-bb98-1bed69ca452a",
      "key": 0,
      "index": 0,
      "source_location": {
        "filename": "882574176.py",
        "filepath": "/private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_16491/882574176.py",
        "line": 6
      },
      "value": "Context: "
    },
    {
      "type": "interpolation",
      "id": "e385617e-8d6d-429d-a32c-42b4f2f996dc",
      "parent_id": "c048387c-480d-4bc8-bb98-1bed69ca452a",
      "key": "ctx",
      "index": 1,
      "source_location": {
        "filename": "882574176.py",
        "filepath": "/private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_16491/882574176.py",
        "line": 6
      },
      "expression": "context",
      "conversion": null,
      "format_spec": "ctx",
      "render_hints": "",
      "value": "User is

## Nested JSON Export

JSON export works recursively for nested prompts.

In [7]:
# Create nested structure
inner = prompt(t"{context:ctx}")
outer = prompt(t"Outer: {inner:in}. {instructions:inst}")

# Export nested structure
nested_json = outer.toJSON()
print(json.dumps(nested_json, indent=2))

{
  "prompt_id": "67192841-f03b-4cbd-9b63-2a4691946732",
  "children": [
    {
      "type": "static",
      "id": "e1496a4c-c613-4132-a992-a7f299f6505d",
      "parent_id": "67192841-f03b-4cbd-9b63-2a4691946732",
      "key": 0,
      "index": 0,
      "source_location": {
        "filename": "3458326394.py",
        "filepath": "/private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_16491/3458326394.py",
        "line": 3
      },
      "value": "Outer: "
    },
    {
      "type": "nested_prompt",
      "id": "912a9ca7-acfb-4a59-871f-a52c6fe894d6",
      "parent_id": "67192841-f03b-4cbd-9b63-2a4691946732",
      "key": "in",
      "index": 1,
      "source_location": {
        "filename": "3458326394.py",
        "filepath": "/private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_16491/3458326394.py",
        "line": 3
      },
      "expression": "inner",
      "conversion": null,
      "format_spec": "in",
      "render_hints": "",
      "prompt_id": "912a9

## Inspecting Prompt Structure

You can recursively inspect nested prompt structures.

In [8]:
from t_prompts import StructuredPrompt


def inspect_prompt(p, indent=0):
    """Recursively inspect a structured prompt."""
    prefix = "  " * indent
    for key, node in p.items():
        if isinstance(node, StructuredPrompt):
            # Node is a nested prompt
            print(f"{prefix}{key}: (nested prompt)")
            inspect_prompt(node, indent + 1)
        else:
            # Node is an interpolation with a string value
            print(f"{prefix}{key}: {node.expression} = \"{node.value}\"")

inspect_prompt(outer)

in: (nested prompt)
  ctx: context = "User is Alice"
inst: instructions = "Be concise"


## Next Steps

Continue learning:

- **03-ir-visualization.ipynb**: See the widget visualization for IntermediateRepresentation
- **topics/few-shot-prompts.ipynb**: Build few-shot prompts with dynamic keys and lists
- **topics/source-mapping.ipynb**: Bidirectional text ↔ structure mapping
- **topics/dedenting.ipynb**: Write readable multi-line prompts